## Sgkit GWAS Workflow Example - Chunk Report

Chunk report for the GWAS workflow that shows the shape and chunk size for each intermediate variable in the calculation.

In [266]:
%run setup.ipynb

In [267]:
%run matmul.ipynb

In [268]:
def create_dataset(n, m, c):
    rs = da.random.RandomState(0)
    XL, BL = rs.randint(0, 128, size=(n, m), chunks=(163, 11584)), da.array([1] + [0] * (m - 1))
    XC, BC = rs.normal(size=(m, c)), rs.normal(size=(c,))
    Y = (XL * BL).sum(axis=0) + XC @ BC + rs.normal(scale=.001, size=m)
    ds = xr.Dataset(dict(
        # This is a proxy for discretized allele dosages (between 0 and 2)
        XL=(('variants', 'samples'), (2 * XL / 127).astype('f2')),
        # This value represents covariates for samples, e.g. age, sex, ancestry, etc.
        XC=(('samples', 'covariates'), XC.astype('f4')),
        # This is the outcome on which all variant data will be regressed separately
        Y=(('samples', 'outcomes'), Y[:, np.newaxis].astype('f4')),
    ))
    return ds

In [269]:
# Choose dataset size
# Debug settings
n = 500 # Number of variants (i.e. genomic locations)
m = 1000 # Number of individuals (i.e. people)
c = 3    # Number of covariates (i.e. confounders)

# Representative settings for single (small) UK Biobank chromosome:
# n, m, c = 141910, 365941, 25

# XY chromosome
n, m, c = 8444, 365941, 25
# factor to multiply XY chromosome variants by (to measure scaling)
# note that chr11 is approx factor=16
factor = 8
n = n * factor

In [270]:
ds = create_dataset(n, m, c)

For comparison, here is the `gwas` function that is broken down below to show shapes and chunk sizes for each intermediate variable.

In [271]:
def gwas(XL, XC, Y):
    # Add intercept
    XC = da.concatenate([da.ones((XC.shape[0], 1), dtype=XC.dtype), XC], axis=1)
    
    # Rechunk along short axes
    XC = XC.rechunk((None, -1))
    Y = Y.rechunk((None, -1))
    dof = Y.shape[0] - XC.shape[1] - 1
    
    # Apply orthogonal projection to eliminate core covariates
    #XLP = XL - XC @ da.linalg.lstsq(XC, XL)[0]
    #YP = Y - XC @ da.linalg.lstsq(XC, Y)[0]
    XLP = XL - matmul(XC, da.linalg.lstsq(XC, XL)[0])
    YP = Y - matmul(XC, da.linalg.lstsq(XC, Y)[0])

    # Estimate coefficients for each loop covariate
    XLPS = (XLP ** 2).sum(axis=0, keepdims=True).T
    #B = (XLP.T @ YP) / XLPS
    B = matmul(XLP.T, YP) / XLPS

    # Compute residuals for each loop covariate and outcome separately
    YR = YP[:, np.newaxis, :] - XLP[..., np.newaxis] * B[np.newaxis, ...]
    RSS = (YR ** 2).sum(axis=0)
    
    # Get t-statistics for coefficient estimates and match to p-values
    T = B / np.sqrt(RSS / dof / XLPS)
    P = da.map_blocks(
        lambda t: 2 * stats.distributions.t.sf(np.abs(t), dof), T, dtype="float64"
    )
    return xr.Dataset(dict(
        beta=(('variants','outcomes'), B), 
        pval=(('variants','outcomes'), P)
    ))
# Define the GWAS regressions
dsr = gwas(
    # Note: This (the largest) array needs to be rechunked due to scalability 
    # issues with da.matmul, specifically https://github.com/dask/dask/pull/6924.
    # See here for more details:
    # https://github.com/pystatgen/sgkit/issues/390#issuecomment-730660134
    ds.XL.data.rechunk((652, 5792)).T.astype('f4'), 
    ds.XC.data, 
    ds.Y.data
)

In [272]:
XL = ds.XL.data
XL = XL.rechunk((652, 5792))
XL

dask.array<rechunk-merge, shape=(67552, 365941), dtype=float16, chunksize=(652, 5792), chunktype=numpy.ndarray>

In [273]:
XL = XL.T.astype('f4')
XL

dask.array<astype, shape=(365941, 67552), dtype=float32, chunksize=(5792, 652), chunktype=numpy.ndarray>

In [274]:
XC = ds.XC.data
XC = da.concatenate([da.ones((XC.shape[0], 1), dtype=XC.dtype), XC], axis=1) # Add intercept
XC = XC.rechunk((None, -1))
XC

dask.array<rechunk-merge, shape=(365941, 26), dtype=float32, chunksize=(365941, 26), chunktype=numpy.ndarray>

In [275]:
# Apply orthogonal projection to eliminate core covariates
LS = da.linalg.lstsq(XC, XL)[0]
#LS = LS.rechunk((None, 326))
LS

dask.array<solve-triangular, shape=(26, 67552), dtype=float32, chunksize=(26, 652), chunktype=numpy.ndarray>

In [276]:
XC @ LS

dask.array<matmul, shape=(365941, 67552), dtype=float32, chunksize=(365941, 652), chunktype=numpy.ndarray>

In [277]:
XLP = XL - XC @ LS
XLP

dask.array<sub, shape=(365941, 67552), dtype=float32, chunksize=(5792, 652), chunktype=numpy.ndarray>

In [278]:
# Estimate coefficients for each loop covariate
XLPS = (XLP ** 2).sum(axis=0, keepdims=True).T
XLPS

dask.array<transpose, shape=(67552, 1), dtype=float32, chunksize=(652, 1), chunktype=numpy.ndarray>

In [279]:
Y = ds.Y.data
Y = Y.rechunk((None, -1))
Y

dask.array<astype, shape=(365941, 1), dtype=float32, chunksize=(11584, 1), chunktype=numpy.ndarray>

In [280]:
# Apply orthogonal projection to eliminate core covariates
YP = Y - matmul(XC, da.linalg.lstsq(XC, Y)[0])
YP

dask.array<sub, shape=(365941, 1), dtype=float32, chunksize=(11584, 1), chunktype=numpy.ndarray>

In [281]:
XLP.T @ YP

dask.array<matmul, shape=(67552, 1), dtype=float32, chunksize=(652, 1), chunktype=numpy.ndarray>

In [282]:
B = (XLP.T @ YP) / XLPS
B

dask.array<truediv, shape=(67552, 1), dtype=float32, chunksize=(652, 1), chunktype=numpy.ndarray>

In [283]:
YR = YP[:, np.newaxis, :] - XLP[..., np.newaxis] * B[np.newaxis, ...]
YR

,Array,Chunk
Bytes,98.88 GB,15.11 MB
Shape,"(365941, 67552, 1)","(5792, 652, 1)"
Count,209887 Tasks,6656 Chunks
Type,float32,numpy.ndarray


In [284]:
RSS = (YR ** 2).sum(axis=0)
RSS

dask.array<sum-aggregate, shape=(67552, 1), dtype=float32, chunksize=(652, 1), chunktype=numpy.ndarray>

In [285]:
dof = Y.shape[0] - XC.shape[1] - 1
dof

365914

In [286]:
T = B / np.sqrt(RSS / dof / XLPS)
T

dask.array<truediv, shape=(67552, 1), dtype=float64, chunksize=(652, 1), chunktype=numpy.ndarray>

In [287]:
P = da.map_blocks(
        lambda t: 2 * stats.distributions.t.sf(np.abs(t), dof), T, dtype="float64"
    )
P

dask.array<lambda, shape=(67552, 1), dtype=float64, chunksize=(652, 1), chunktype=numpy.ndarray>

In [292]:
# approx guess of which arrays need to be kept in memory at once
arrs = [XL, XLP, YR]
total_mem = sum([arr.nbytes for arr in arrs])
dask.utils.format_bytes(total_mem)

'296.64 GB'

In [293]:
# approx how many workers do we need for this amount of memory in the cluster?
n1_standard_8_mem = 30 * 1_000_000_000
total_mem // n1_standard_8_mem

9